In [1]:
import pandas as pd
import numpy as np

from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cosine

c:\Users\ethan\.conda\envs\nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = SentenceTransformer('all-MiniLM-L6-v2')

c:\Users\ethan\.conda\envs\nlp\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
df = pd.read_csv("../data/okcupid_profiles.csv")
#df_sample = df.sample(1000).reset_index(drop = True)
df_sample = df.copy()


In [4]:
df_sample.columns

Index(['age', 'status', 'sex', 'orientation', 'body_type', 'diet', 'drinks',
       'drugs', 'education', 'ethnicity', 'height', 'income', 'job',
       'last_online', 'location', 'offspring', 'pets', 'religion', 'sign',
       'smokes', 'speaks', 'essay0', 'essay1', 'essay2', 'essay3', 'essay4',
       'essay5', 'essay6', 'essay7', 'essay8', 'essay9'],
      dtype='object')

In [5]:
# All essays but essay9, which is "you should message me if..."

df_demographics = df_sample[df_sample.columns.drop(list(df_sample.filter(regex="essay")))]
essays_df = df_sample.loc[:, ["essay0", "essay1", "essay2", "essay3", "essay4", 
                   "essay5", "essay6", "essay7", "essay8"]]
essays_df = essays_df.fillna(" ").astype(str)

essays_df.loc[:, "all_essays"] = essays_df.apply(" ".join, axis = 1)

df_all = pd.concat([df_demographics, essays_df.loc[:, ["all_essays"]]], axis = 1)

In [1]:
embedding_array = np.load('sentence_transformer_embeddings.npy')

NameError: name 'np' is not defined

In [13]:
def compute_cosine_similarity(target_vector, vectors):
    similarities = []
    for vector in vectors:
        similarity = 1 - cosine(target_vector, vector)  # 1 - cosine distance to get cosine similarity
        similarities.append(similarity)
    return similarities

In [44]:
from itertools import islice

def take(n, iterable):
    """Return the first n items of the iterable as a list."""
    return list(islice(iterable, n))

In [50]:
def rank_matches(input_bio, pref_gender=False, pref_age_lower=False, pref_age_higher=False, top_n=5):
    df_possible = df_all.copy()
    if pref_gender:
        df_possible = df_possible.loc[df_possible.loc[:,'sex'] == pref_gender, :]
    if pref_age_higher:
        df_possible = df_possible[df_possible.loc[:, "age"] <= pref_age_higher]
    if pref_age_lower:
        df_possible = df_possible[df_possible.loc[:, "age"] >= pref_age_lower]


    user_embeddings = model.encode(input_bio)

    other_embeddings = [embedding_array[i] for i in df_possible.index]
    # Compute the cosine similarity between the user's weighted embedding vector and all possible matches
    cosine_similarities = compute_cosine_similarity(user_embeddings, other_embeddings)
    # Recover index to match back to original dataframe
    similarity_scores = {index:score for index, score in enumerate(cosine_similarities)}
    # Sort by similarity
    ranked_similarity = dict(sorted(similarity_scores.items(), key=lambda item: item[1], reverse = True))

    top_match = take(5, ranked_similarity.items())

    bios = [essays_df.loc[t[0], "all_essays"] for t in top_match]


    return bios

In [51]:
test = rank_matches("Test this for app usage")

In [52]:
test

['new to this app. what do you think?                ',
 'just checking this app out                ',
 '  test              ',
 'created for a/b testing. more data later.                ',
 'i am trying this to see if my phone can help me find a match with this pof app. if it cannot i will blame the phone.  i am some italian, some belgian with a nice sense of humor. i am well traveled. i go to europe once a year. i have an accent. i recently came here from chicago. i like poetry. you could say i am romantic and sexy lol working hard...playing hard              ']